## Практика Python

В данном разделе мы выступим в роли data scientist и попытаемся построить простую модель для рекомендации фильмов пользователям.

Загрузите в колаб файлы по оценкам (ratings) и фильмам (movies) и создайте на их основе pandas-датафреймы

In [1]:
import pandas as pd

In [2]:
df_rating = pd.read_csv('ml-100k/u.data.csv', sep='\t', names = ['user_id', 'item_id', 'rating', 'timestamp'])
df_rating.head(3)

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116


In [3]:
df_movie = pd.read_csv('u.item.csv', encoding='cp866', sep='|', names = ['movie id', 'movie title', 'release date', 'video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'])
df_movie.head(3)

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Childrens,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [4]:
df_movie['year'] = pd.to_datetime(df_movie['release date']).dt.year
df_movie.head(3)

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Childrens,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,year
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,1995.0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,1995.0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1995.0


Средствами Pandas, используя dataframe ratings, найдите id пользователя, поставившего больше всего оценок

In [5]:
df_rating_max = df_rating.groupby('user_id').count().sort_values(by='rating', ascending=False).head(1).reset_index()
df_rating_max

,user_id,item_id,rating,timestamp
0,405,737,737,737


In [6]:
user_max = df_rating_max['user_id']
user_max

0    405
Name: user_id, dtype: int64

In [7]:
type(user_max)

pandas.core.series.Series

In [8]:
#перевод Series в число
user_id = sum(user_max)
user_id

405

In [9]:
type(user_id)

int

Оставьте в датафрейме ratings только те фильмы, который оценил данный пользователь 

In [10]:
df_rating_filter = df_rating[df_rating['user_id'] == user_id]
df_rating_filter.head()

,user_id,item_id,rating,timestamp
12276,405,56,4,885544911
12383,405,592,1,885548670
12430,405,1582,1,885548670
12449,405,171,1,885549544
12460,405,580,1,885547447


Добавьте к датафрейму из задания 3 столбцы:
 - По жанрам. Каждый столбец - это жанр. Единицу записываем, если фильм принадлежит данному жанру и 0 - если нет

In [11]:
table_join = df_rating_filter.merge(df_movie, 
    left_on='item_id', right_on='movie id', how='left').drop(['user_id', 'timestamp', 'movie id', 'movie title', 'video release date', 'IMDb URL'], axis=1)
table_join.head(3)

,item_id,rating,release date,unknown,Action,Adventure,Animation,Childrens,Comedy,Crime,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,year
0,56,4,01-Jan-1994,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1994.0
1,592,1,01-Jan-1995,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,1995.0
2,1582,1,01-Jan-1947,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1947.0


In [12]:
# создадим датафрейм с кол-вом оценок и суммой оценок по каждому фильму
df_rating_movie = df_rating.groupby('item_id').agg(['count', 'sum'])['rating'].reset_index()
df_rating_movie.head(3)

,item_id,count,sum
0,1,452,1753
1,2,131,420
2,3,90,273


Добавьте к датафрейму из задания 3 столбцы:
 - столбцы с общим количеством оценок от всех пользователей на фильм и суммарной оценкой от всех пользователей

In [13]:
table_join2 = table_join.merge(df_rating_movie, on='item_id', how='left')
table_join2.head(3)

,item_id,rating,release date,unknown,Action,Adventure,Animation,Childrens,Comedy,Crime,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,year,count,sum
0,56,4,01-Jan-1994,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1994.0,394,1600
1,592,1,01-Jan-1995,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,1995.0,9,30
2,1582,1,01-Jan-1947,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1947.0,1,1


Теперь все готово и можно строить модель

Сформируйте X_train, X_test, y_train, y_test

In [14]:
X, y = table_join2[['year', 'unknown', 'Action', 'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western', 'count', 'sum']], table_join2['rating']

In [15]:
# Функции для разделения данных на данные для обучения и тестирования
from sklearn.model_selection import train_test_split

In [16]:
# Разделяем данные на данные для обучения и тестирования
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Возьмите модель линейной регрессии (или любую другую для задачи регрессии)  и обучите ее на фильмах

**линейная регрессия**

In [17]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [18]:
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [19]:
model.predict(X_test)

array([ 1.22821728e+00,  2.83699138e+00,  1.33823085e+00,  1.55024710e+00,
        1.97553742e+00,  1.47833088e+00,  1.29275706e+00,  2.46275502e+00,
        1.55137484e+00,  1.08677277e+00,  1.16664615e+00,  1.75948343e+00,
        1.77396902e+00,  1.25103654e+00,  1.40754199e+00,  3.12831533e+00,
        1.03986649e+00,  1.52643371e+00,  2.57852163e+00,  3.98427377e+00,
        2.96139461e+00,  1.35077305e+00,  1.49419605e+00,  3.46608181e+00,
        1.34970814e+00,  1.28796560e+00,  1.21457806e+00,  3.34394951e+00,
        2.54708604e+00,  2.07178923e+00,  3.47422922e+00,  2.63626945e+00,
        1.36482379e+00,  1.21061245e+00,  1.54306415e+00,  3.65914889e+00,
        1.19724819e+00,  1.31075273e+00,  1.22713612e+00,  1.10891923e+00,
        1.42054065e+00,  1.81187965e+00,  1.98139271e-01,  1.63310378e+00,
        1.06502127e+00,  1.28821933e+00,  1.66295808e+00,  2.31246243e+00,
        2.86914623e+00,  1.89088010e+00,  3.65875360e+00,  1.26711840e+00,
        3.75854902e+00,  

In [38]:
result_lin = pd.DataFrame([y_test.values, model.predict(X_test)]).T
result_lin.head(3)

,0,1
0,1.0,1.228217
1,5.0,2.836991
2,1.0,1.338231


**логистическая регрессия**

In [21]:
from sklearn.linear_model import LogisticRegression

In [22]:
lr = LogisticRegression()

In [23]:
lr.fit(X_train, y_train)

C:\Users\trufanovdv\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [39]:
result_log = pd.DataFrame([y_test.values, lr.predict(X_test)]).T
result_log.head(3)

,0,1
0,1,1
1,5,1
2,1,1


**классификатор дерева решений**

In [24]:
from sklearn.tree import DecisionTreeClassifier

In [25]:
model_tree = DecisionTreeClassifier()

In [26]:
model_tree.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

Оцените качество модели на X_test, y_test при помощи метрик для задачи регрессии

In [27]:
from sklearn.metrics import mean_squared_error

In [28]:
# точность логистической регрессии по MSE
mean_squared_error(y_test, lr.predict(X_test))

2.0337837837837838

In [29]:
sqrt_lr = mean_squared_error(y_test, lr.predict(X_test))**0.5
sqrt_lr

1.426107914494476

In [30]:
# точность дерева решений по MSE
mean_squared_error(y_test, model_tree.predict(X_test))

2.858108108108108

In [31]:
sqrt_tree = mean_squared_error(y_test, model_tree.predict(X_test))**0.5
sqrt_tree

1.6905940104318682

In [32]:
# точность линейной регрессии по MSE
mean_squared_error(y_test, model.predict(X_test))

1.0914159851296283

In [33]:
sqrt_line = mean_squared_error(y_test, model.predict(X_test))**0.5
sqrt_line

1.0447085646866443

In [34]:
from sklearn.metrics import mean_absolute_error

In [35]:
# точность логистической регрессии по MAE
mean_absolute_error(y_test, lr.predict(X_test))

0.7094594594594594

In [36]:
# точность дерева решений по MAE
mean_absolute_error(y_test, model_tree.predict(X_test))

1.0202702702702702

In [37]:
# точность линейной регрессии по MAE
mean_absolute_error(y_test, model.predict(X_test))

0.8192805688663183

как по MSE так и по MAE наименьшую точность показала модель дерева решений. Ее отбрасываем.

выбираем самую оптимальную модель между линейной и логистичекой регрессией.  
 - Среднеквадратическое отклонение наименьшее у модели, построенной по линейной регрессии (1.09), MAE - 0.819  
 - Абсолютное отклоение наименьшее у модели, построенной по логистической регрессии (0.709), MSE - 2.034  
 
Скорее всего логистическая регрессия дала более близкие результаты, но имеет ряд выбросов, за счет чего MSE большое.  
Проверим эту гипотезу на отклонениях

In [41]:
result_lin['dif'] = result_lin[0] - result_lin[1]
result_lin.head(3)

,0,1,dif
0,1.0,1.228217,-0.228217
1,5.0,2.836991,2.163009
2,1.0,1.338231,-0.338231


In [43]:
result_lin.describe()

,0,1,dif
count,148.000000,148.000000,148.000000
mean,1.804054,1.952435,-0.148381
std,1.333293,0.860527,1.037629
min,1.000000,-0.002138,-2.984274
25%,1.000000,1.315351,-0.735852
50%,1.000000,1.707523,-0.332052
75%,2.000000,2.451298,0.453919
max,5.000000,4.465994,3.934979


In [42]:
result_log['dif'] = result_log[0] - result_log[1]
result_log.head(3)

,0,1,dif
0,1,1,0
1,5,1,4
2,1,1,0


In [44]:
result_log.describe()

,0,1,dif
count,148.000000,148.000000,148.000000
mean,1.804054,1.608108,0.195946
std,1.333293,1.431593,1.417379
min,1.000000,1.000000,-4.000000
25%,1.000000,1.000000,0.000000
50%,1.000000,1.000000,0.000000
75%,2.000000,1.000000,0.000000
max,5.000000,5.000000,4.000000


На мой взгляд, наиболее точной явлется модель Линейной регрессии, т.к. среднее отклонение меньше (-0,148), меньше разница между min и max